In [9]:
import pandas as pd
from geopy.geocoders import Nominatim
import geopandas as gpd

In [2]:
listings_df = pd.read_csv("resources/merged_cleaned_ny.csv", encoding="ISO-8859-1")

In [3]:
listings_df.columns

Index(['primary_key', 'listing_id', 'listing_url', 'name', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count', 'latitude',
       'longitude', 'room_type', 'accommodates', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'first_review', 'last_review', 'review_scores_rating',
       'calculated_host_listings_count', 'reviews_per_month', 'date',
       'available', 'price', 'adjusted_price'],
      dtype='object')

In [4]:
# Function to get neighborhood and borough from latitude and longitude
def get_neighborhood_borough(lat, lon):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.reverse((lat, lon), language="en")
    address = location.raw['address']
    neighborhood = address.get('neighbourhood', '')
    borough = address.get('borough', '')
    return neighborhood, borough


In [6]:
unique_listings_df = listings_df.copy()
unique_listings_df = unique_listings_df[['listing_id', 'latitude', 'longitude']]
unique_listings_df = unique_listings_df.drop_duplicates(keep = 'first')
print(len(unique_listings_df))
unique_listings_df.head()

313


,listing_id,latitude,longitude
0,2595,40.753560,-73.985590
32,5121,40.685350,-73.955120
64,14991,40.722070,-73.989760
96,5136,40.662650,-73.994540
128,15341,40.721359,-73.993668


In [7]:
five_listings_df = unique_listings_df.copy()
five_listings_df = five_listings_df.head(5)
five_listings_df.head(10)

,listing_id,latitude,longitude
0,2595,40.753560,-73.985590
32,5121,40.685350,-73.955120
64,14991,40.722070,-73.989760
96,5136,40.662650,-73.994540
128,15341,40.721359,-73.993668


In [8]:
# Add 'Neighborhood' and 'Borough' columns to DataFrame
five_listings_df['neighborhood'], five_listings_df['borough'] = zip(*five_listings_df.apply(lambda row: get_neighborhood_borough(row['latitude'], row['longitude']), axis=1))


,listing_id,latitude,longitude,neighborhood,borough
0,2595,40.753560,-73.985590,Manhattan Community Board 5,
32,5121,40.685350,-73.955120,,
64,14991,40.722070,-73.989760,Manhattan Community Board 3,
96,5136,40.662650,-73.994540,Sunset Park,
128,15341,40.721359,-73.993668,Manhattan Community Board 2,


In [10]:
# Fetching New York City's boroughs and neighborhoods using geopandas
ny_boroughs = gpd.read_file(gpd.datasets.get_path('nybb'))
ny_neighborhoods = gpd.read_file(gpd.datasets.get_path('nybb')).to_crs(epsg=4326)


In [11]:
# Function to find borough and neighborhood based on latitude and longitude
def find_neighborhood_borough(lat, lon):
    point = gpd.GeoSeries([Point(lon, lat)], crs='EPSG:4326')
    borough = None
    neighborhood = None

    for index, borough_polygon in ny_boroughs.iterrows():
        if borough_polygon['geometry'].contains(point.geometry[0]):
            borough = borough_polygon['BoroName']

    for index, neighborhood_polygon in ny_neighborhoods.iterrows():
        if neighborhood_polygon['geometry'].contains(point.geometry[0]):
            neighborhood = neighborhood_polygon['Name']

    return neighborhood, borough


In [12]:
# Add 'Neighborhood' and 'Borough' columns to DataFrame based on spatial data
five_listings_df['neighborhood'], five_listings_df['borough'] = zip(*five_listings_df.apply(lambda row: find_neighborhood_borough(row['latitude'], row['longitude']), axis=1))
five_listings_df.head()

NameError: name 'Point' is not defined